In [1]:
import pennylane as qml
from pennylane import numpy as np

np.random.seed(42)

In [15]:
n_wires = 8
graph = [(0,1), (0,2), (1,2), (1,3), (1,4), (2,3), (3,5), (4,5), (4,6), (5,6), (5,7), (6,7)]

# unitary operator U_B with parameter beta
def U_B(beta):
    for wire in range(n_wires):
        qml.RX(2 * beta, wires=wire)


# unitary operator U_C with parameter gamma
def U_C(gamma):
    for edge in graph:
        wire1 = edge[0]
        wire2 = edge[1]
        qml.CNOT(wires=[wire1, wire2])
        qml.RZ(gamma, wires=wire2)
        qml.CNOT(wires=[wire1, wire2])

In [3]:
def bitstring_to_int(bit_string_sample):
    bit_string = "".join(str(bs) for bs in bit_string_sample)
    return int(bit_string, base=2)

In [4]:
dev = qml.device("default.qubit", wires=n_wires, shots=1)

In [5]:
@qml.qnode(dev)
def circuit(gammas, betas, edge=None, n_layers=1):
    # apply Hadamards to get the n qubit |+> state
    for wire in range(n_wires):
        qml.Hadamard(wires=wire)
    # p instances of unitary operators
    for i in range(n_layers):
        U_C(gammas[i])
        U_B(betas[i])
    if edge is None:
        # measurement phase
        return qml.sample()
    # during the optimization phase we are evaluating a term
    # in the objective using expval
    H = qml.PauliZ(edge[0]) @ qml.PauliZ(edge[1])
    return qml.expval(H)

In [6]:
def hamiltonian(config, graph):
    
    H = 0
    for i, j in graph:
        
        zi = config[i]
        zj = config[j]
        H += 0.5 * (1 - zi*zj)
    
    return H

In [7]:
def qaoa_maxcut(n_layers=1):
    print("\np={:d}".format(n_layers))

    # initialize the parameters near zero
    init_params = 0.01 * np.random.rand(2, n_layers, requires_grad=True)

    # minimize the negative of the objective function
    def objective(params):
        gammas = params[0]
        betas = params[1]
        neg_obj = 0
        for edge in graph:
            # objective for the MaxCut problem
            neg_obj -= 0.5 * (1 - circuit(gammas, betas, edge=edge, n_layers=n_layers))
        return neg_obj

    # initialize optimizer: Adagrad works well empirically
    opt = qml.AdagradOptimizer(stepsize=0.5)

    # optimize parameters in objective
    params = init_params
    steps = 30
    for i in range(steps):
        params = opt.step(objective, params)
        if (i + 1) % 5 == 0:
            print("Objective after step {:5d}: {: .7f}".format(i + 1, -objective(params)))

    # sample measured bitstrings 100 times
    bit_strings = []
    n_samples = 100
    for i in range(0, n_samples):
        bit_strings.append(bitstring_to_int(circuit(params[0], params[1], edge=None, n_layers=n_layers)))

    simple_measurements = []
    for i in range(0, n_samples):
        measurements = circuit(params[0], params[1], edge=None, n_layers=n_layers)
        simple_measurements.append(measurements)
        bit_strings.append(bitstring_to_int(measurements))
        
    samples_energies = []
    for m in simple_measurements:
        m = m.tolist()
        m = [int(x) if int(x)==1 else -1 for x in m]
        samples_energies.append(hamiltonian(m, graph))
       
    average_energy = sum(samples_energies)/len(samples_energies)
    print('average energy:', average_energy)

    # print optimal parameters and most frequently sampled bitstring
    counts = np.bincount(np.array(bit_strings))
    most_freq_bit_string = np.argmax(counts)
    print("Optimized (gamma, beta) vectors:\n{}".format(params[:, :n_layers]))
    print("Most frequently sampled bit string is: {:04b}".format(most_freq_bit_string))

    return -objective(params), bit_strings


# perform qaoa on our graph with p=1,2 and
# keep the bitstring sample lists
bitstrings1 = qaoa_maxcut(n_layers=1)[1]
bitstrings2 = qaoa_maxcut(n_layers=2)[1]

#bitstring_1 = qaoa_maxcut(n_layers=10)[1]


p=1
Objective after step     5:  6.0000000
Objective after step    10:  8.0000000
Objective after step    15:  2.0000000
Objective after step    20:  7.0000000
Objective after step    25:  12.0000000
Objective after step    30:  10.0000000
average energy: 8.99
Optimized (gamma, beta) vectors:
[[-0.57419498]
 [-1.13981379]]
Most frequently sampled bit string is: 1011001

p=2
Objective after step     5:  8.0000000
Objective after step    10:  6.0000000
Objective after step    15:  12.0000000
Objective after step    20:  9.0000000
Objective after step    25:  9.0000000
Objective after step    30:  11.0000000
average energy: 9.54
Optimized (gamma, beta) vectors:
[[-0.14412807  0.53606249]
 [ 0.52176739 -0.23004518]]
Most frequently sampled bit string is: 1001001


In [8]:
bitstring_4 = qaoa_maxcut(n_layers = 4)


p=4


Objective after step     5:  10.0000000
Objective after step    10:  10.0000000
Objective after step    15:  7.0000000
Objective after step    20:  8.0000000
Objective after step    25:  11.0000000
Objective after step    30:  9.0000000
average energy: 9.37
Optimized (gamma, beta) vectors:
[[-2.08552366  1.86530964  0.74742751  0.12741789]
 [-1.74464598  0.26295831 -0.06723413  1.40977286]]
Most frequently sampled bit string is: 10111100


In [9]:
bitstring_10 = qaoa_maxcut(n_layers=10)


p=10
Objective after step     5:  7.0000000
Objective after step    10:  8.0000000
Objective after step    15:  6.0000000
Objective after step    20:  8.0000000
Objective after step    25:  8.0000000
Objective after step    30:  13.0000000
average energy: 7.9
Optimized (gamma, beta) vectors:
[[ 0.18461719 -0.5517497   1.10068752 -1.10629094 -1.2940881  -1.81337606
  -0.60096275 -0.23573089  0.30085027 -0.62771566]
 [-0.21005528 -0.11541473 -0.1059826  -0.12856344  0.81998585 -0.56805769
  -0.67260841  2.25891814  1.37887079 -1.11792429]]
Most frequently sampled bit string is: 10100000


In [10]:
bitstring_4_1 = qaoa_maxcut(n_layers=4)


p=4
Objective after step     5:  7.0000000
Objective after step    10:  7.0000000
Objective after step    15:  9.0000000
Objective after step    20:  10.0000000
Objective after step    25:  12.0000000
Objective after step    30:  10.0000000
average energy: 10.2
Optimized (gamma, beta) vectors:
[[ 0.2204381  -0.03398016  0.31138846  0.38423113]
 [ 1.37148908 -1.52903492  1.33970446 -0.33821546]]
Most frequently sampled bit string is: 1001001


In [12]:
# Ladder
bitstring_4_2 = qaoa_maxcut(n_layers=4)


p=4
Objective after step     5:  6.0000000
Objective after step    10:  5.0000000
Objective after step    15:  5.0000000
Objective after step    20:  7.0000000
Objective after step    25:  10.0000000
Objective after step    30:  10.0000000
average energy: 7.97
Optimized (gamma, beta) vectors:
[[-0.12559526 -0.55373576 -0.32343085  0.35095845]
 [ 0.34453939 -1.17754165  0.33476463 -0.42793725]]
Most frequently sampled bit string is: 1010101


In [14]:
#Barbell
bitstring_4_3 = qaoa_maxcut(n_layers=4)


p=4
Objective after step     5:  6.0000000
Objective after step    10:  6.0000000
Objective after step    15:  6.0000000
Objective after step    20:  9.0000000
Objective after step    25:  3.0000000
Objective after step    30:  8.0000000
average energy: 7.17
Optimized (gamma, beta) vectors:
[[-0.85779981 -1.43385171  1.0828236  -0.6194914 ]
 [ 0.98138594 -1.42312912 -0.75459556 -0.71207399]]
Most frequently sampled bit string is: 110110


In [16]:
#Caveman
bitstring_10_1 = qaoa_maxcut(n_layers=4)


p=4
Objective after step     5:  6.0000000
Objective after step    10:  6.0000000
Objective after step    15:  5.0000000
Objective after step    20:  8.0000000
Objective after step    25:  7.0000000
Objective after step    30:  5.0000000
average energy: 6.34
Optimized (gamma, beta) vectors:
[[ 1.37203897 -1.17075919 -2.17614946 -1.91137934]
 [-0.88349023  0.1636271   0.71956344 -0.11144558]]
Most frequently sampled bit string is: 1011100


In [ ]:
bitstring_10_2 = qaoa_maxcut(n_layers=10)

In [ ]:
bitstring_10_3 = qaoa_maxcut(n_layers=10)